# Introduction to Python  

### [Decorators](https://realpython.com/primer-on-python-decorators/)

Decorators are functions that modify other functions’ behaviors without changing their core operations. As indicated by the name, decorators only decorate other functions.  

#### Functions

Before you can understand decorators, you must first understand how functions work. For our purposes, a function returns a value based on the given arguments. Here is a very simple example:

In [1]:
def add_one(number):
    return number + 1

In [2]:
add_one(2)

3

#### First-Class Objects

In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on). Consider the following three functions:

In [3]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [4]:
greet_bob(say_hello)

'Hello Bob'

In [5]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

#### Inner Functions

It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [6]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [7]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


#### Returning Functions From Functions

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function:

In [8]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

In [9]:
first = parent(1)
second = parent(2)

print(first)
print(second)

<function parent.<locals>.first_child at 0x7fd4a01f4bf8>
<function parent.<locals>.second_child at 0x7fd4a01f4d08>


In [10]:
first()

'Hi, I am Emma'

In [11]:
second()

'Call me Liam'

#### Simple Decorators

Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [11]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [12]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


#### Explanation  

To understand what’s going on here, look back at the previous examples. We are literally just applying everything you have learned so far.

The so-called decoration happens at the following line:

say_whee = my_decorator(say_whee)

In effect, the name say_whee now points to the wrapper() inner function. Remember that you return wrapper as a function when you call my_decorator(say_whee):

In [13]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

However, wrapper() has a reference to the original say_whee() as func, and calls that function between the two calls to print().

Put simply: decorators wrap a function, modifying its behavior.

Before moving on, let’s have a look at a second example. Because wrapper() is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [14]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

In [17]:
say_whee()

Whee!


Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the “pie” syntax. The following example does the exact same thing as the first decorator example:

In [18]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

So, @my_decorator is just an easier way of saying say_whee = my_decorator(say_whee). It’s how you apply a decorator to a function.

In [19]:
#Create a file called decorators.py with the following content:

def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

#from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

say_whee()

Whee!
Whee!


#### Decorating Functions With Arguments

Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:

In [20]:
@do_twice
def greet(name):
    print(f"Hello {name}")

In [21]:
greet("World")  #error

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function wrapper_do_twice() does not take any arguments, but name="World" was passed to it. You could fix this by letting wrapper_do_twice() accept one argument, but then it would not work for the say_whee() function you created earlier.

The solution is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite decorators.py as follows:


In [22]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [23]:
@do_twice
def greet(name):
    print(f"Hello {name}")

The wrapper_do_twice() inner function now accepts any number of arguments and passes them on to the function it decorates. Now both your say_whee() and greet() examples works:

In [24]:
say_whee()

Whee!
Whee!


In [25]:
greet("World")

Hello World
Hello World


### Returning Values From Decorated Functions

What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:


In [26]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [27]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


In [28]:
print(hi_adam)

None


Oops, your decorator ate the return value from the function.

Because the do_twice_wrapper() doesn’t explicitly return a value, the call return_greeting("Adam") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorator to:


In [29]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [30]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [31]:
print(return_greeting("Adam"))

Creating greeting
Creating greeting
Hi Adam


#### Who Are You, Really?

A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability. Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:


In [32]:
print

<function print>

In [33]:
print.__name__

'print'

In [34]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



The introspection works for functions you define yourself as well:

In [35]:
say_whee

<function __main__.do_twice.<locals>.wrapper_do_twice()>

In [36]:
say_whee.__name__

'wrapper_do_twice'

However, after being decorated, say_whee() has gotten very confused about its identity. It now reports being the wrapper_do_twice() inner function inside the do_twice() decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the @functools.wraps decorator, which will preserve information about the original function. Update your decorator again:

In [37]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [38]:
@do_twice
def say_whee():
    print("Whee!")

In [39]:
say_whee

<function __main__.say_whee()>

In [40]:
say_whee.__name__

'say_whee'

In [41]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



### The decorator template

In [41]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator


### Decorators with [Arguments](https://medium.com/better-programming/python-decorators-5-advanced-features-to-know-17dd9be7517b)

In [42]:
import time
from functools import wraps

def logging_time(unit):
    """Decorator that logs time"""
    def logger(func):
        @wraps(func)
        def inner_logger(*args, **kwargs):
            """Function that logs time"""
            start = time.time()
            func(*args, **kwargs)
            scaling = 1000 if unit == "ms" else 1
            print(f"Calling {func.__name__}: {(time.time() - start) * scaling:.5f} {unit}")
        return inner_logger
    return logger

In [43]:
@logging_time("ms")
def calculate_sum_ms(n):
    """Calculate sum of 0 to n-1"""
    return sum(range(n))

@logging_time("s")
def calculate_sum_s(n):
    """Calculate sum of 0 to n-1"""
    return sum(range(n))

calculate_sum_ms(100000)
calculate_sum_s(100000)

Calling calculate_sum_ms: 1.65248 ms
Calling calculate_sum_s: 0.00175 s


### Multiple decorators

In [44]:
def repeat(func):
    """Decorator that repeats function call twice"""
    def repeater(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return repeater

In [45]:
@logging_time("ms")
@repeat
def say_hi(whom):
    print(f"Hi, {whom}!")

In [46]:
@repeat
@logging_time("ms")
def say_hello(whom):
    print(f"Hello, {whom}!")

In [47]:
say_hi("John")

Hi, John!
Hi, John!
Calling repeater: 0.14925 ms


In [48]:
say_hello("Aaron")

Hello, Aaron!
Calling say_hello: 0.07200 ms
Hello, Aaron!
Calling say_hello: 0.02909 ms


### Class-Based decorators 

In [49]:
class Repeat:
    def __init__(self, n):
        self.n = n

    def __call__(self, func):
        def repeater(*args, **kwargs):
            for _ in range(self.n):
                func(*args, **kwargs)
        return repeater

In [50]:
@Repeat(n=2)
def morning_greet(person):
    print(f"Good Morning, {person}!")

In [51]:
@Repeat(n=3)
def afternoon_greet(person):
    print(f"Good Afternoon, {person}!")

In [52]:
morning_greet("Jason")

Good Morning, Jason!
Good Morning, Jason!


In [53]:
afternoon_greet("Kelly")

Good Afternoon, Kelly!
Good Afternoon, Kelly!
Good Afternoon, Kelly!


### Examples

#### Timing Functions

Let’s start by creating a @timer decorator. It will measure the time a function takes to execute and print the duration to the console. Here’s the code:

In [54]:
import functools
import time

In [55]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])


In [56]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0026 secs


In [57]:
waste_some_time(100)

Finished 'waste_some_time' in 0.2757 secs


#### Slowing Down Code

This next example might not seem very useful. Why would you want to slow down your Python code? Probably the most common use case is that you want to rate-limit a function that continuously checks whether a resource—like a web page—has changed. The @slow_down decorator will sleep one second before it calls the decorated function:


In [58]:
def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [59]:
countdown(3)

3
2
1
Liftoff!


### Advanced Topics  

<ul>
<li><a href="#decorating-classes">Decorators on classes</a></li>
<li><a href="#nesting-decorators">Several decorators on one function</a></li>
<li><a href="#decorators-with-arguments">Decorators with arguments</a></li>
<li><a href="#both-please-but-never-mind-the-bread">Decorators that can optionally take arguments</a></li>
<li><a href="#stateful-decorators">Stateful decorators</a></li>
<li><a href="#classes-as-decorators">Classes as decorators</a></li>
</ul>